In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pystan

%matplotlib inline
custom_style = {'axes.labelcolor': 'white',
                'xtick.color': 'white',
                'ytick.color': 'white'}
sns.set_style("darkgrid", rc=custom_style)
sns.set_context("notebook")
plt.style.use('dark_background')
plt.rcParams["font.size"] = 18
np.random.seed(123)

In [68]:
df = pd.read_csv('./data/ch6_2_1.csv', header=None)
df = df.T
df.columns = ['x1', 'x2']
# x1 = np.array(df['x1']).astype(np.float)
# x2 = np.array(df['x2']).astype(np.float)

In [75]:
stan_model = """
    data{
        int N1;
        int N2;
        real X1[N1];
        real X2[N2];
    }
    
    parameters{
        real mu1;
        real mu2;
        real<lower=0> sigma1;
        real<lower=0> sigma2;
    }
    
    model{
        for(n in 1:N1){
            X1[n] ~ normal(mu1, sigma1);
        }
        for(n in 1:N2){
            X2[n] ~ normal(mu2, sigma2);
        }
#        X1 ~ normal(mu1, sigma1);
#        X2 ~ normal(mu2, sigma2);

    }
    
    generated quantities{
        real mu_delta;
        real<lower=0, upper=1> u_0;
        real<lower=0, upper=1> u_1;

        mu_delta <- mu2 - mu1;
        u_0 <- step(mu_delta - 0);
        u_1 <- step(mu_delta - 1);

    }
"""

In [76]:
sm = pystan.StanModel(model_code = stan_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cf283ac7c025d4139df6406f5496966f NOW.


In [77]:
stan_data = {"N1": df.shape[0], "N2": df.shape[0], "X1": df['x1'], "X2": df['x2']}

In [78]:
fit = sm.sampling(data=stan_data, iter=11000, warmup=1000, chains=3, seed=1234)

/Users/yugi/.pyenv/versions/anaconda-4.0.0/envs/py36/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [79]:
fit

Inference for Stan model: anon_model_cf283ac7c025d4139df6406f5496966f.
3 chains, each with iter=11000; warmup=1000; thin=1; 
post-warmup draws per chain=10000, total post-warmup draws=30000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu1       30.84  1.6e-3   0.25  30.35  30.68  30.84   31.0  31.33  23016    1.0
mu2       31.98  2.4e-3   0.39  31.21  31.73  31.98  32.23  32.75  27019    1.0
sigma1     1.09  1.2e-3    0.2   0.79   0.95   1.06    1.2   1.55  24619    1.0
sigma2     1.74  2.0e-3   0.31   1.26   1.52    1.7   1.92   2.45  23852    1.0
mu_delta   1.14  2.9e-3   0.46   0.23   0.84   1.14   1.45   2.06  25759    1.0
u_0        0.99  5.8e-4   0.09    1.0    1.0    1.0    1.0    1.0  22428    1.0
u_1        0.62  2.8e-3   0.49    0.0    0.0    1.0    1.0    1.0  30000    1.0
lp__     -30.58    0.01   1.46 -34.29 -31.31 -30.25  -29.5 -28.74  12562    1.0

Samples were drawn using NUTS at Sun Aug 12 09:38:54 2018.
For each parameter, n_eff is